In [1]:
from gensim.models import Word2Vec
from gensim import models
import numpy as np
import MeCab
import pandas as pd
#from gensim.models import word2vec, KeyedVectors
mt = MeCab.Tagger('')
mt.parse('')

ModuleNotFoundError: No module named 'gensim'

In [ ]:
## Just for e

In [1]:
from gensim.models.word2vec import LineSentence

inp = '/home/iftekhar/myworkplace/AI-system/2VECs_models/word2vec/mobicontrol_corpus.txt'
var = LineSentence(inp)

for sentence in var:
    print(sentence)
    break
    

In [58]:
# we can use the commented following for *.model files otherwise use keyedvectors
gensim_model = Word2Vec.load('word2vec/combined.model')
# import gensim.models.keyedvectors as word2vec
# model = word2vec.KeyedVectors.load_word2vec_format('word2vec/combined.bin', binary=True)

In [59]:
gensim_model.wv.most_similar ("こんにちわ") 
# for element in model.most_similar("ありがとう"):
#      print(element [0] , element [1])

[('さようなら', 0.6923430562019348),
 ('こんにちは', 0.6762766242027283),
 ('やぁ', 0.6219089031219482),
 ('おやすみなさい', 0.6146820187568665),
 ('母さん', 0.6144062280654907),
 ('はじめまして', 0.6134737730026245),
 ('おかあさん', 0.6094452738761902),
 ('ありがとう', 0.6041801571846008),
 ('お父さん', 0.5938694477081299),
 ('おいで', 0.5913469791412354)]

In [61]:
gensim_model.wv.most_similar ("mobicontrol") 

[('soti', 0.7755337953567505),
 ('MCDisp', 0.7417123317718506),
 ('MCLink', 0.7239100933074951),
 ('HKEYLOCALMACHINE', 0.7149759531021118),
 ('Name', 0.7014230489730835),
 ('href', 0.7005501985549927),
 ('Launch', 0.6990233659744263),
 ('Local', 0.6928770542144775),
 ('System', 0.691757082939148),
 ('Camera', 0.6907753944396973)]

In [37]:
s1="こんにちわ"
s2="ありがとう"
distance = gensim_model.wv.n_similarity(s1.lower().split(), s2.lower().split())
distance

0.59298605

In [62]:
def get_vector(text):
    sum_vec = np.zeros(200)
    word_count = 0
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        # 名詞、動詞、形容詞に限定
        if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
            try: 
                temp = gensim_model.wv[node.surface]
            except KeyError:
                temp = 0
            sum_vec += temp
            word_count += 1
        node = node.next
    return sum_vec / word_count

# cos類似度を計算
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

v1 = get_vector('昨日、お笑い番組を見た。')
v2 = get_vector('昨夜、テレビで漫才をやっていた。')
v3 = get_vector('昨日、公園に行った。')

print(cos_sim(v1, v2))
print(cos_sim(v1, v3))

0.6332798224169414
0.18655876518845044


In [63]:
print(cos_sim(get_vector("mobicontrol settings"), get_vector("wifi settings")))

0.6057180479829244


In [57]:
gensim_model.wv.most_similar("wifi")

[('テザリング', 0.7307436466217041),
 ('umts', 0.6860231161117554),
 ('dtcp', 0.6787638664245605),
 ('felica', 0.6723511815071106),
 ('mobile', 0.6712178587913513),
 ('mhl', 0.6692694425582886),
 ('bluetooth', 0.6688016653060913),
 ('airh', 0.6648038029670715),
 ('モデム', 0.6639145612716675),
 ('hw', 0.6635802388191223)]

In [39]:
df = pd.read_csv("/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/varieties_corpus/processed_word5perline_corpus.csv")
data = df.Text.values.tolist()

In [37]:
df = pd.read_csv("/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/varieties_corpus/processed_perPage_perLine.csv")
df.Data = df.Data.apply(lambda x: clean_text(str(x)))
data = df.Data.values.tolist()
cleaned_data = deep_clean(data)

In [36]:
 def deep_clean(data):
    clean_data = []
    for items in ''.join(data).split():
        if len(items) < 2:
            items = re.sub(r'[ぁ-ん]', '', items)
            items = re.sub(r'[ア-ン]', '', items)
            items = re.sub(r'[A-Za-z]', '', items)
            # print(items)
            clean_data.append(items)
        else:
            clean_data.append(items)
    return [cleaned for cleaned in clean_data if cleaned not in ['']]

In [ ]:
clean_data[0:500]

In [26]:
with open("corpus.txt", "w") as outfile:
     outfile.write("\n".join(str(item) for item in clean_data))


In [10]:
import re
def clean_text(text):
    replaced = text.replace("\\","")
    replaced = replaced.replace("+","")
    replaced = re.sub('\W+',' ', replaced)
    replaced = re.sub(r'￥', '', replaced)       # 【】の除去
    replaced = re.sub(r'．', '', replaced)       # ・ の除去
    replaced = re.sub(r'｣', '', replaced)     # （）の除去
    replaced = re.sub(r'｢', '', replaced)   # ［］の除去
    replaced = re.sub(r'～', '', replaced)  # メンションの除去
    replaced = re.sub(r'｜', '', replaced)  # URLの除去
    replaced = re.sub(r'＠', '', replaced)  # 全角空白の除去
    replaced = re.sub(r'？', '', replaced) # 数字の除去
    replaced = re.sub(r'％', '', replaced)
    replaced = re.sub(r'＝', '', replaced)
    replaced = re.sub(r'！', '', replaced)
    replaced = re.sub(r'｝', '', replaced)
    replaced = re.sub(r'：', '', replaced)
    replaced = re.sub(r'－', '', replaced)
    replaced = re.sub(r'･', '', replaced)
    replaced = re.sub(r'ｔ', '', replaced)
    replaced = re.sub(r'ｋ', '', replaced)
    replaced = re.sub(r'ｄ', '', replaced)
    return replaced


### Old work below, will work later

In [1]:
df = pd.read_csv("processed.csv")


In [2]:
inp = "ja.text.seg" 
outp1 = "ja.text.model" 
outp2 = "ja.text.vector"
out3 = "ja.text.bin"

model = Word2Vec(data, min_count=1,size=200, window=5, sg=1, workers=multiprocessing.cpu_count(), iter=10)
#model.save(outp1)
#model.wv.save_word2vec_format(outp2, binary=False)
#model.wv.save_word2vec_format(out3, binary=True)

In [3]:
myinput = input("Enter Query String: ")
s = myinput.lower().split()

Enter Query String: database setting


In [175]:
row_saver=[]
index_saver=[]
distance_saver=[]
d=0model = Word2Vec(data, min_count=1,size=200, window=5, sg=1, workers=multiprocessing.cpu_count(), iter=10)

for i in range(len(df)):
    g = only_data.values[i]
    distance = model.wv.n_similarity(s,g)
    #print(distance)
    if d<distance:
        d=distance
        page_index = df.iloc[i,-1]
        distance_saver.append(distance)
        row_saver.append(i)
        index_saver.append(page_index)

In [176]:
mydf=pd.DataFrame(list(zip(index_saver,distance_saver,row_saver)),
                  columns=['page_index','distance','Row_index'])

In [177]:
allT = []
for i in mydf.Row_index.values:
    #print(df.iloc[[i]].values)
    allT.append(list(df.iloc[i,1:-1].values.ravel()))

In [178]:
AllText = pd.DataFrame()
AllText['Text'] = allT

In [179]:
mydf['Text'] = AllText['Text']

In [180]:
mydf.head()

,page_index,distance,Row_index,Text
0,0,0.370014,0,"[MobiControl, の, 設定, 順序, 端末, グループ, の, 作成, コンソー..."
1,0,0.406018,1,"[の, 接続, プロファイル, を, 作成, し, て, おく, 全て, の, 端末, OS..."
2,0,0.448790,17,"[Inc, Canada, Translated, by, Penetrate, of, L..."
3,2,0.464922,71,"[の, 移動, ルール, データ, 収集, ルール, アラートルール, 通信, 費, 管理,..."
4,2,0.482667,93,"[の, インストール, と, サーバ, へ, の, 登録, 方法, について, は, 下記,..."


In [ ]:
mydf.Text=mydf.Text.apply(lambda x: ''.join(str(x).strip("[]")))
mydf

0.85121775

In [101]:
#load word2vec model, here GoogleNews is used
Binmodel = gensim.models.KeyedVectors.load_word2vec_format('ja.text.bin', binary=True)
#two sample sentences 
s1 = 'database setting'

row_saver=[]
index_saver=[]
distance_saver=[]
d=0
for i in range(len(df)):
    g = only_data.values[i]
    #calculate distance between two sentences using WMD algorithm
    distance = Binmodel.wmdistance(s1, g)
    #print(distance)
    if d<distance:
        d=distance
        page_index = df.iloc[i,-1]
        distance_saver.append(distance)
        row_saver.append(i)
        index_saver.append(page_index)
        
print ('distance = %.3f' % distance)

distance = 0.924


In [108]:
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess    

model = gensim.models.Word2Vec.load("ja.text.model")

def tidy_sentence(sentence, vocabulary):
    return [word for word in simple_preprocess(sentence) if word in vocabulary]    

def compute_sentence_similarity(sentence_1, sentence_2, model_wv):
    vocabulary = set(model_wv.index2word)    
    tokens_1 = tidy_sentence(sentence_1, vocabulary)    
    tokens_2 = tidy_sentence(sentence_2, vocabulary)    
    return model_wv.n_similarity(tokens_1, tokens_2)

wv = KeyedVectors.load('ja.text.model', mmap='r')
sim = compute_sentence_similarity('this is a sentence', 'this is also a sentence', model.wv)
print(sim)

0.99999994


In [50]:
model.similarity('database','google')

/home/ifte/amiebot_project/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.746703

In [69]:
print(model)

Word2Vec(vocab=5940, size=50, alpha=0.025)
